# Primer Prototipo
Se implementa una primera versión funcional del algoritmo para construir bloques a partir de combinaciones de transacciones, evaluando su utilidad y validez.

In [2]:
# Imports
import pandas as pd
import numpy as np
from itertools import combinations

## 1. Simulación de datos

In [3]:
# Simulamos 6 transacciones 
data = [
    {"tx_id": "t1", "gas": 10000, "fee": 60, "from": "A", "to": "X", "depends_on": None},
    {"tx_id": "t2", "gas": 30000, "fee": 120, "from": "B", "to": "X", "depends_on": None},
    {"tx_id": "t3", "gas": 15000, "fee": 110, "from": "C", "to": "X", "depends_on": None},
    {"tx_id": "t4", "gas": 5000,  "fee": 50,  "from": "D", "to": "Y", "depends_on": "t1"},
    {"tx_id": "t5", "gas": 2000,  "fee": 20,  "from": "E", "to": "Z", "depends_on": None},
    {"tx_id": "t6", "gas": 25000, "fee": 90,  "from": "B", "to": "X", "depends_on": None},  # conflicto con t2 (mismo from)
]

df = pd.DataFrame(data)
df

,tx_id,gas,fee,from,to,depends_on
0,t1,10000,60,A,X,None
1,t2,30000,120,B,X,None
2,t3,15000,110,C,X,None
3,t4,5000,50,D,Y,t1
4,t5,2000,20,E,Z,None
5,t6,25000,90,B,X,None


## 2. Generación de combinaciones de a pares

In [4]:
# Generamos todas las combinaciones posibles de a pares
combos = list(combinations(df.to_dict("records"), 2))
print(f"Combinaciones: {combos}")
print(f"Total de combinaciones: {len(combos)}")

Combinaciones: [({'tx_id': 't1', 'gas': 10000, 'fee': 60, 'from': 'A', 'to': 'X', 'depends_on': None}, {'tx_id': 't2', 'gas': 30000, 'fee': 120, 'from': 'B', 'to': 'X', 'depends_on': None}), ({'tx_id': 't1', 'gas': 10000, 'fee': 60, 'from': 'A', 'to': 'X', 'depends_on': None}, {'tx_id': 't3', 'gas': 15000, 'fee': 110, 'from': 'C', 'to': 'X', 'depends_on': None}), ({'tx_id': 't1', 'gas': 10000, 'fee': 60, 'from': 'A', 'to': 'X', 'depends_on': None}, {'tx_id': 't4', 'gas': 5000, 'fee': 50, 'from': 'D', 'to': 'Y', 'depends_on': 't1'}), ({'tx_id': 't1', 'gas': 10000, 'fee': 60, 'from': 'A', 'to': 'X', 'depends_on': None}, {'tx_id': 't5', 'gas': 2000, 'fee': 20, 'from': 'E', 'to': 'Z', 'depends_on': None}), ({'tx_id': 't1', 'gas': 10000, 'fee': 60, 'from': 'A', 'to': 'X', 'depends_on': None}, {'tx_id': 't6', 'gas': 25000, 'fee': 90, 'from': 'B', 'to': 'X', 'depends_on': None}), ({'tx_id': 't2', 'gas': 30000, 'fee': 120, 'from': 'B', 'to': 'X', 'depends_on': None}, {'tx_id': 't3', 'gas': 150

## 3. Función de utilidad, penalizaciones y bonificaciones

In [5]:
def eval_pair(tx1, tx2):
    total_fee = tx1["fee"] + tx2["fee"]
    total_gas = tx1["gas"] + tx2["gas"]
    penalty = 0
    bonus = 0

    # Penalización por conflicto (mismo remitente)
    if tx1["from"] == tx2["from"]:
        penalty += 50
    # Penalización por dependencia no cumplida 
    if tx2["depends_on"] == tx1["tx_id"]:
        bonus += 10
    elif tx1["depends_on"] == tx2["tx_id"]:
        bonus += 10
    elif tx1["depends_on"] or tx2["depends_on"]:
        # dependencia no resuelta
        penalty += 20

    utilidad = total_fee + bonus - penalty
    return {
        "pair": (tx1["tx_id"], tx2["tx_id"]),
        "fee": total_fee,
        "gas": total_gas,
        "penalty": penalty,
        "bonus": bonus,
        "utilidad": utilidad
    }

# Evaluar todas las combinaciones
results = [eval_pair(tx1, tx2) for tx1, tx2 in combos]
pd.DataFrame(results).sort_values(by="utilidad", ascending=False)

,pair,fee,gas,penalty,bonus,utilidad
5,"(t2, t3)",230,45000,0,0,230
11,"(t3, t6)",200,40000,0,0,200
0,"(t1, t2)",180,40000,0,0,180
1,"(t1, t3)",170,25000,0,0,170
8,"(t2, t6)",210,55000,50,0,160
6,"(t2, t4)",170,35000,20,0,150
4,"(t1, t6)",150,35000,0,0,150
9,"(t3, t4)",160,20000,20,0,140
7,"(t2, t5)",140,32000,0,0,140
10,"(t3, t5)",130,17000,0,0,130


# 4. Función para verificar validez de un par 

In [6]:
def is_valid_pair(tx1, tx2):
    if tx1["from"] == tx2["from"]:
        return False, "Conflicto: mismo 'from'"
    if tx2["depends_on"] == tx1["tx_id"]:
        return True, "Dependencia cumplida"
    if tx1["depends_on"] == tx2["tx_id"]:
        return False, "Dependencia invertida"
    if tx1["depends_on"] and tx1["depends_on"] != tx2["tx_id"]:
        return False, "Dependencia rota"
    if tx2["depends_on"] and tx2["depends_on"] != tx1["tx_id"]:
        return False, "Dependencia rota"
    return True, "Válida"



# 5. Verificamos validez de las combinaciones 

In [7]:
for result in results:
    tx1 = next(tx for tx in data if tx["tx_id"] == result["pair"][0])
    tx2 = next(tx for tx in data if tx["tx_id"] == result["pair"][1])
    valid, reason = is_valid_pair(tx1, tx2)
    result["valid"] = valid
    result["reason"] = reason

df_results = pd.DataFrame(results)
df_results.sort_values(by="utilidad", ascending=False)


,pair,fee,gas,penalty,bonus,utilidad,valid,reason
5,"(t2, t3)",230,45000,0,0,230,True,Válida
11,"(t3, t6)",200,40000,0,0,200,True,Válida
0,"(t1, t2)",180,40000,0,0,180,True,Válida
1,"(t1, t3)",170,25000,0,0,170,True,Válida
8,"(t2, t6)",210,55000,50,0,160,False,Conflicto: mismo 'from'
6,"(t2, t4)",170,35000,20,0,150,False,Dependencia rota
4,"(t1, t6)",150,35000,0,0,150,True,Válida
9,"(t3, t4)",160,20000,20,0,140,False,Dependencia rota
7,"(t2, t5)",140,32000,0,0,140,True,Válida
10,"(t3, t5)",130,17000,0,0,130,True,Válida


# 6. Construcción del Bloque

In [8]:
# límite de gas del bloque
GAS_LIMIT = 40000

# Inicializar variables
used_txs = set()
gas_used = 0
block = []

# Ordenar por utilidad descendente y filtrar válidas
valid_pairs = [r for r in results if r["valid"]]
valid_pairs.sort(key=lambda x: x["utilidad"], reverse=True)

# Iterar y agregar combinaciones no conflictivas
for pair in valid_pairs:
    tx_a, tx_b = pair["pair"]
    if tx_a in used_txs or tx_b in used_txs:
        continue
    if gas_used + pair["gas"] > GAS_LIMIT:
        continue
    block.append(pair)
    used_txs.update([tx_a, tx_b])
    gas_used += pair["gas"]

# Mostrar resultado del bloque
print("Bloque construido:")
for b in block:
    print(f"  {b['pair']} → utilidad: {b['utilidad']}, gas: {b['gas']}")
    
print(f"\nGas total usado: {gas_used}")
print(f"Utilidad total del bloque: {sum(b['utilidad'] for b in block)}")


Bloque construido:
  ('t3', 't6') → utilidad: 200, gas: 40000

Gas total usado: 40000
Utilidad total del bloque: 200
